In [5]:
import pandas as pd

In [6]:
food_fndds = pd.read_parquet('fndds_parquet/food.parquet')
food_nutrient_fndds = pd.read_parquet('fndds_parquet/food_nutrient.parquet')
nutrient_fndds = pd.read_parquet('fndds_parquet/nutrient.parquet').set_index('nutrient_nbr')
nutrient_fndds = nutrient_fndds.loc[nutrient_fndds.index.notna()]
# nutrient_fndds.drop(columns=['id', 'rank']).to_parquet('fndds_parquet/nutrient.parquet')
# food_fndds.to_parquet('fndds_parquet/food.parquet')
# food_nutrient_fndds.to_parquet('fndds_parquet/food_nutrient.parquet')
# nutrient_fndds.to_parquet('fndds_parquet/nutrient.parquet')

In [7]:
food_legacy = pd.read_parquet('legacy_parquet/food.parquet')
food_nutrient_legacy = pd.read_parquet('legacy_parquet/food_nutrient.parquet')
nutrient_legacy = pd.read_parquet('legacy_parquet/nutrient.parquet')
id_to_nbd = pd.read_parquet('legacy_parquet/id_to_nbd.parquet')
f_ndb = food_legacy.join(id_to_nbd, how='left').reset_index().set_index('NDB_number')
# food_legacy.to_parquet('legacy_parquet/food.parquet')
# food_nutrient_legacy.to_parquet('legacy_parquet/food_nutrient.parquet')
# nutrient_legacy.to_parquet('legacy_parquet/nutrient.parquet')
# id_to_nbd.to_parquet('legacy_parquet/id_to_nbd.parquet')

In [8]:
codes = pd.read_csv('food_codes.csv', encoding='latin1')
codes = codes.loc[codes.price.notna()]
codes_legacy = codes.loc[codes.fndds_code.isna()].copy()
codes_fndds = codes.loc[codes.fndds_code.notna()].copy()
codes_fndds['fndds_code'] = codes_fndds.fndds_code.astype(int)
codes_legacy = codes_legacy.merge(f_ndb, left_on='ndb_code', right_index=True)
codes_legacy = codes_legacy[['food', 'name', 'price', 'lbs', 'multiplier', 'fdc_id', 'description']]
codes_fndds = codes_fndds.merge(food_fndds.reset_index(), left_on='fndds_code', right_on='food_code')
codes_fndds = codes_fndds[['food', 'name', 'price', 'lbs', 'multiplier', 'fdc_id', 'description']]
codes_fndds = codes_fndds.rename(columns={'food_code': 'fdc_id'})
codes = pd.concat([codes_legacy, codes_fndds]).set_index('fdc_id')

In [9]:
food_nutrient = pd.concat([food_nutrient_fndds, food_nutrient_legacy])
core_nutrients = codes.join(food_nutrient, how='left')

In [10]:
core_nutrients.columns

Index(['food', 'name', 'price', 'lbs', 'multiplier', 'description', 'id',
       'nutrient_id', 'amount', 'data_points', 'derivation_id', 'min', 'max',
       'median', 'footnote', 'min_year_acquired'],
      dtype='object')

In [11]:
core_nutrients['nutrient_id'] = core_nutrients.nutrient_id.astype(float)
core_nutrients = core_nutrients.drop(columns='name')

In [12]:
cnap = core_nutrients.merge(nutrient_fndds.reset_index(), left_on='nutrient_id', right_on='nutrient_nbr', how='inner')
cna = cnap.pivot(index='food', columns=['name','nutrient_nbr'], values='amount')
cna.to_parquet('core_nutrient_amounts_v1.parquet')
cna.droplevel(1, axis=1).sort_values(by='Energy')

name,SFA 6:0,"Vitamin B-12, added",Riboflavin,Folic acid,PUFA 20:4,"Vitamin E, added",PUFA 22:5 n-3 (DPA),"Folate, DFE",MUFA 20:1,"Fatty acids, total monounsaturated",...,"Selenium, Se","Calcium, Ca","Iron, Fe",Vitamin E (alpha-tocopherol),Total lipid (fat),Protein,"Potassium, K",Vitamin D (D2 + D3),Energy,"Vitamin A, RAE"
food,,,,,,,,,,,,,,,,,,,,,
Bok Choy,0.0,0.0,0.070,0.0,0.000,0.0,0.0,66.0,0.000,0.015,...,0.5,105.0,0.80,0.09,0.20,1.50,252.0,0.0,13.0,223.0
Lettuce,0.0,0.0,0.053,0.0,0.000,0.0,0.0,34.0,0.000,0.006,...,0.4,27.0,0.64,0.20,0.14,1.13,168.0,0.0,14.0,198.0
Cucumber,0.0,0.0,0.033,0.0,0.000,0.0,0.0,7.0,0.000,0.005,...,0.3,16.0,0.28,0.03,0.11,0.65,147.0,0.0,15.0,5.0
Zucchini,0.0,0.0,0.094,0.0,0.000,0.0,0.0,24.0,0.000,0.011,...,0.2,16.0,0.37,0.12,0.32,1.21,261.0,0.0,17.0,10.0
Swiss Chard,0.0,0.0,0.090,0.0,0.000,0.0,0.0,14.0,0.000,0.040,...,0.9,51.0,1.80,1.89,0.20,1.80,379.0,0.0,19.0,306.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Almonds,0.0,0.0,1.197,0.0,0.000,0.0,0.0,55.0,0.007,33.076,...,2.0,268.0,3.73,23.90,52.54,20.96,713.0,0.0,598.0,0.0
Hazelnuts,0.0,0.0,0.113,0.0,0.000,0.0,0.0,113.0,0.131,45.652,...,2.4,114.0,4.70,15.03,60.75,14.95,680.0,0.0,628.0,1.0
Walnuts,0.0,0.0,0.150,0.0,0.000,0.0,0.0,98.0,0.134,8.933,...,4.9,98.0,2.91,0.70,65.21,15.23,441.0,0.0,654.0,1.0


In [13]:
cnap['price_per_lb'] = cnap.price / cnap.lbs
cnap['multiplier'] = cnap.multiplier.fillna(1)
cnap['price_per_lb_adjusted'] = cnap.price / (cnap.lbs * cnap.multiplier)
cnap = cnap.loc[:,['food', 'price', 'lbs', 'multiplier', 'description', 'amount',
             'nutrient_nbr', 'name',
       'unit_name', 'price_per_lb', 'price_per_lb_adjusted']]
cnap.to_parquet('core_nutrient_amounts_prices_v1.parquet')

In [14]:
cnap.name.unique()

array(['SFA 6:0', 'Vitamin B-12, added', 'Riboflavin', 'Folic acid',
       'PUFA 20:4', 'Vitamin E, added', 'PUFA 22:5 n-3 (DPA)',
       'Folate, DFE', 'MUFA 20:1', 'Fatty acids, total monounsaturated',
       'Choline, total', 'PUFA 22:6 n-3 (DHA)', 'PUFA 18:4', 'MUFA 16:1',
       'SFA 12:0', 'SFA 18:0', 'PUFA 20:5 n-3 (EPA)',
       'Vitamin K (phylloquinone)', 'PUFA 18:2',
       'Fatty acids, total polyunsaturated', 'MUFA 22:1', 'Cholesterol',
       'MUFA 18:1', 'Folate, food', 'Folate, total', 'SFA 4:0',
       'SFA 16:0', 'Vitamin B-12', 'Fatty acids, total saturated',
       'Niacin', 'SFA 14:0', 'SFA 10:0', 'SFA 8:0', 'Vitamin B-6',
       'PUFA 18:3', 'Phosphorus, P', 'Zinc, Zn', 'Sugars, Total', 'Water',
       'Cryptoxanthin, beta', 'Theobromine', 'Fiber, total dietary',
       'Sodium, Na', 'Carotene, alpha', 'Thiamin', 'Carotene, beta',
       'Copper, Cu', 'Magnesium, Mg', 'Lutein + zeaxanthin',
       'Vitamin C, total ascorbic acid', 'Retinol', 'Alcohol, ethyl',
   

In [15]:
def ppnu(df, nutrient):
    return df.loc[df.name == nutrient].sort_values(by='price_per_nutrient_unit')

In [16]:
cnap.unit_name.unique()
LB_CONVERSIONS = {
    'G': 0.00220462,
    'MG': 0.00000220462,
    'UG': 0.00000000220462,
}

In [17]:
cnap['g'] = cnap.lbs * 453.592
cnap['price_per_g'] = cnap.price / cnap.g
cnap['price_per_g_adjusted'] = cnap.price / (cnap.g * cnap.multiplier)
cnap['price_per_100_g'] = cnap.price_per_g * 100
cnap['price_per_100_g_adjusted'] = cnap.price_per_g_adjusted * 100
cnap['price_per_nutrient_unit'] = cnap.price_per_100_g_adjusted / cnap.amount

In [18]:
ppnu(cnap, 'PUFA 20:5 n-3 (EPA)')

,food,price,lbs,multiplier,description,amount,nutrient_nbr,name,unit_name,price_per_lb,price_per_lb_adjusted,g,price_per_g,price_per_g_adjusted,price_per_100_g,price_per_100_g_adjusted,price_per_nutrient_unit
4105,Salmon,8.00,1.00,0.75,"Fish, salmon, NFS",0.860,629.0,PUFA 20:5 n-3 (EPA),G,8.00000,10.666667,453.592,0.017637,0.023516,1.763700,2.351599,2.734418
4180,Pink Salmon,3.60,1.00,1.00,"Fish, salmon, canned",0.274,629.0,PUFA 20:5 n-3 (EPA),G,3.60000,3.600000,453.592,0.007937,0.007937,0.793665,0.793665,2.896587
4420,Mussels,10.00,1.00,1.00,Mussels,0.238,629.0,PUFA 20:5 n-3 (EPA),G,10.00000,10.000000,453.592,0.022046,0.022046,2.204624,2.204624,9.263128
4391,Oysters,10.00,1.00,1.00,"Oysters, raw",0.177,629.0,PUFA 20:5 n-3 (EPA),G,10.00000,10.000000,453.592,0.022046,0.022046,2.204624,2.204624,12.455505
4242,Shrimp,7.50,1.00,0.75,"Shrimp, NFS",0.074,629.0,PUFA 20:5 n-3 (EPA),G,7.50000,10.000000,453.592,0.016535,0.022046,1.653468,2.204624,29.792222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1562,Potatoes,4.39,8.00,0.75,"Potato, NFS",0.000,629.0,PUFA 20:5 n-3 (EPA),G,0.54875,0.731667,3628.736,0.001210,0.001613,0.120979,0.161305,inf
1553,Broccoli,1.29,0.75,1.00,"Broccoli, raw",0.000,629.0,PUFA 20:5 n-3 (EPA),G,1.72000,1.720000,340.194,0.003792,0.003792,0.379195,0.379195,inf
1470,Cabbage,0.75,1.00,1.00,"Cabbage, green, raw",0.000,629.0,PUFA 20:5 n-3 (EPA),G,0.75000,0.750000,453.592,0.001653,0.001653,0.165347,0.165347,inf
1979,Oranges,4.29,3.00,1.00,"Orange, raw",0.000,629.0,PUFA 20:5 n-3 (EPA),G,1.43000,1.430000,1360.776,0.003153,0.003153,0.315261,0.315261,inf


In [19]:
cnap.to_parquet('core_nutrient_amounts_prices_v2.parquet')

In [20]:
constraints= pd.read_csv('nutrient_constraints_csv.csv')
cne = constraints.loc[constraints.target.notna() | constraints.ll.notna() | constraints.ul.notna()]

In [22]:
cne.head()

,nutrient_nbr,id,name,unit_name,rank,target,ll,ul
0,203,1003,Protein,G,600,NaN,62.500000,218.750000
1,204,1004,Total lipid (fat),G,800,NaN,55.555556,97.222222
2,205,1005,"Carbohydrate, by difference",G,1110,NaN,281.250000,406.250000
3,208,1008,Energy,KCAL,300,2500.0,NaN,NaN
6,262,1057,Caffeine,MG,18300,NaN,NaN,400.000000


In [25]:
cnap['nutrient_unit_per_dollar'] = cnap.amount / cnap.price_per_100_g_adjusted

In [26]:
cnap.head()

,food,price,lbs,multiplier,description,amount,nutrient_nbr,name,unit_name,price_per_lb,price_per_lb_adjusted,g,price_per_g,price_per_g_adjusted,price_per_100_g,price_per_100_g_adjusted,price_per_nutrient_unit,nutrient_unit_per_dollar
0,Brown rice,21.99,25.0,3.0,"Rice, brown, cooked, no added fat",0.000,608.0,SFA 6:0,G,0.8796,0.2932,11339.8,0.001939,0.000646,0.193919,0.06464,inf,0.000000
1,Brown rice,21.99,25.0,3.0,"Rice, brown, cooked, no added fat",0.000,578.0,"Vitamin B-12, added",UG,0.8796,0.2932,11339.8,0.001939,0.000646,0.193919,0.06464,inf,0.000000
2,Brown rice,21.99,25.0,3.0,"Rice, brown, cooked, no added fat",0.069,405.0,Riboflavin,MG,0.8796,0.2932,11339.8,0.001939,0.000646,0.193919,0.06464,0.936806,1.067457
3,Brown rice,21.99,25.0,3.0,"Rice, brown, cooked, no added fat",0.000,431.0,Folic acid,UG,0.8796,0.2932,11339.8,0.001939,0.000646,0.193919,0.06464,inf,0.000000
4,Brown rice,21.99,25.0,3.0,"Rice, brown, cooked, no added fat",0.000,620.0,PUFA 20:4,G,0.8796,0.2932,11339.8,0.001939,0.000646,0.193919,0.06464,inf,0.000000


In [27]:
cnap.food.unique()

array(['Brown rice', 'Quinoa', 'White Rice', 'Sugar', 'Oats', 'Tomatoes',
       'Cucumber', 'Zucchini', 'Eggplant', 'Red Bell Pepper', 'Peas',
       'Green Bell Pepper', 'Carrots', 'Onions', 'Garlic', 'Spinach',
       'Corn', 'Kale', 'Swiss Chard', 'Bok Choy', 'Brussels Sprouts',
       'Lettuce', 'Cabbage', 'Broccoli', 'Potatoes', 'Sweet Potato',
       'Cauliflower', 'Asparagus', 'Mushrooms', 'Apples', 'Oranges',
       'Grapefruit', 'Pomegranates', 'Bananas', 'Watermelon',
       'Honeydew Melon', 'Cantaloupe', 'Pears', 'Peaches', 'Strawberries',
       'Mango', 'Pineapple', 'Kiwi', 'Raspberries', 'Blueberries',
       'Cherries', 'Almonds', 'Hazelnuts', 'Peanuts', 'Macadamia nuts',
       'Cashews', 'Walnuts', 'Pumpkin seeds', 'Sunflower seeds',
       'Chia seeds', 'Avocados', 'Olive Oil', 'Chicken breast',
       'Chicken thigh', 'Turkey', 'Beef Liver', 'Pork',
       'Beef (grass-fed beef is more nutritious) \x96 regular beef or ground beef',
       'Salmon', 'Pink Salmon', '

In [28]:
cnap.to_csv('core_nutrient_amounts_prices_v3.csv')

In [29]:
cnap.columns

Index(['food', 'price', 'lbs', 'multiplier', 'description', 'amount',
       'nutrient_nbr', 'name', 'unit_name', 'price_per_lb',
       'price_per_lb_adjusted', 'g', 'price_per_g', 'price_per_g_adjusted',
       'price_per_100_g', 'price_per_100_g_adjusted',
       'price_per_nutrient_unit', 'nutrient_unit_per_dollar'],
      dtype='object')